In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import openpyxl

def fetch_crypto_data(crypto_pair, start_date):
    # Set the base URL for the CryptoCompare API
    url = "https://min-api.cryptocompare.com/data/v2/histoday"

    # Split the crypto pair to get the base and quote symbols
    base_currency, quote_currency = crypto_pair.split("/")

    # Convert start_date to timestamp format
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    
    # Initialize parameters
    params = {
        'fsym': base_currency,
        'tsym': quote_currency,
        'limit': 2000,  # Maximum limit of records per request
        'toTs': start_timestamp,
        'api_key': "199d177fdfff4b199ea57ec835e30c7a9e2d2b47321fc87a4672411acb9a5357"  # Replace with your actual CryptoCompare API key
    }
    
    # Make the request to get historical data
    response = requests.get(url, params=params)
    data = response.json()

    # Check if the API call was successful and data exists
    if response.status_code == 200 and data.get("Data") and data["Data"]["Data"]:
        # Parse the data into a DataFrame
        df = pd.DataFrame(data["Data"]["Data"])
        df["time"] = pd.to_datetime(df["time"], unit='s')
        df.rename(columns={
            "time": "Date",
            "open": "Open",
            "high": "High",
            "low": "Low",
            "close": "Close"
        }, inplace=True)
        return df[["Date", "Open", "High", "Low", "Close"]]
    else:
        print("Error fetching data:", data.get("Message", "Unknown error"))
        return pd.DataFrame()  # Return an empty DataFrame on failure





Fetch Cryptocurrency Data from CryptoCompare API
This code fetches historical daily cryptocurrency data for a specified crypto pair (e.g., "BTC/USDT") using the CryptoCompare API. 

In [3]:
# Creating a Python Function to Calcluate Highest , Lowest , Difference Metrics 
def calculate_metrics(data, variable1, variable2):
    # Calculate Historical High Price and Days Since High
    data[f'High_Last_{variable1}_Days'] = data['High'].rolling(window=variable1, min_periods=1).max()
    data[f'Days_Since_High_Last_{variable1}_Days'] = (
        data['High'].rolling(window=variable1, min_periods=1)
        .apply(lambda x: (len(x) - x.argmax() - 1), raw=True)
    )
    data[f'%_Diff_From_High_Last_{variable1}_Days'] = (
        ((data['Close'] - data[f'High_Last_{variable1}_Days']) / data[f'High_Last_{variable1}_Days']) * 100
    )
    
    # Calculate Historical Low Price and Days Since Low
    data[f'Low_Last_{variable1}_Days'] = data['Low'].rolling(window=variable1, min_periods=1).min()
    data[f'Days_Since_Low_Last_{variable1}_Days'] = (
        data['Low'].rolling(window=variable1, min_periods=1)
        .apply(lambda x: (len(x) - x.argmin() - 1), raw=True)
    )
    data[f'%_Diff_From_Low_Last_{variable1}_Days'] = (
        ((data['Close'] - data[f'Low_Last_{variable1}_Days']) / data[f'Low_Last_{variable1}_Days']) * 100
    )
    
    # Calculate Future High Price and % Difference from Future High
    data[f'High_Next_{variable2}_Days'] = data['High'].shift(-variable2).rolling(window=variable2, min_periods=1).max()
    data[f'%_Diff_From_High_Next_{variable2}_Days'] = (
        ((data['Close'] - data[f'High_Next_{variable2}_Days']) / data[f'High_Next_{variable2}_Days']) * 100
    )
    
    # Calculate Future Low Price and % Difference from Future Low
    data[f'Low_Next_{variable2}_Days'] = data['Low'].shift(-variable2).rolling(window=variable2, min_periods=1).min()
    data[f'%_Diff_From_Low_Next_{variable2}_Days'] = (
        ((data['Close'] - data[f'Low_Next_{variable2}_Days']) / data[f'Low_Next_{variable2}_Days']) * 100
    )
    
    return data

From Here Onwards i am initalising 4 diffrent Data Frame to Store the Fetcehd and Processed Data collected from the API's

In [4]:

crypto_pair = "BTC/USD"
start_date = "2023-01-01"
df1 = fetch_crypto_data(crypto_pair, start_date)
df1= calculate_metrics(df1 , 7 , 5)
df1.head()


,Date,Open,High,Low,Close,High_Last_7_Days,Days_Since_High_Last_7_Days,%_Diff_From_High_Last_7_Days,Low_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,%_Diff_From_High_Next_5_Days,Low_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2017-07-10,2511.43,2530.34,2271.79,2344.02,2530.34,0.0,-7.363437,2271.79,0.0,3.179431,2237.13,4.777997,1974.03,18.742876
1,2017-07-11,2344.02,2412.75,2265.48,2324.29,2530.34,1.0,-8.143174,2265.48,0.0,2.595918,2237.13,3.896063,1808.99,28.485508
2,2017-07-12,2324.29,2424.82,2262.08,2403.09,2530.34,2.0,-5.028968,2262.08,0.0,6.233643,2237.13,7.418433,1808.99,32.841530
3,2017-07-13,2403.09,2436.66,2334.96,2362.44,2530.34,3.0,-6.635472,2262.08,1.0,4.436625,2400.74,-1.595341,1808.99,30.594420
4,2017-07-14,2362.44,2370.53,2165.36,2234.17,2530.34,4.0,-11.704751,2165.36,0.0,3.177763,2412.38,-7.387310,1808.99,23.503723


In [5]:
crypto_pair2 = "BTC/USDT"
start_date2 = start_date
df2 = fetch_crypto_data(crypto_pair=crypto_pair2 , start_date= start_date2)
df2 = calculate_metrics(df2 , 7 , 5)
df2.head()


,Date,Open,High,Low,Close,High_Last_7_Days,Days_Since_High_Last_7_Days,%_Diff_From_High_Last_7_Days,Low_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,%_Diff_From_High_Next_5_Days,Low_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2017-07-10,2481.79,2509.38,2243.20,2334.61,2509.38,0.0,-6.964669,2243.20,0.0,4.074982,2220.62,5.133251,1950.52,19.691672
1,2017-07-11,2334.61,2383.42,2133.41,2310.25,2509.38,1.0,-7.935426,2133.41,0.0,8.289077,2220.62,4.036260,1792.65,28.873455
2,2017-07-12,2310.25,2536.68,2220.25,2536.68,2536.68,0.0,0.000000,2133.41,1.0,18.902602,2220.62,14.232962,1792.65,41.504477
3,2017-07-13,2536.68,2536.68,2270.98,2338.29,2536.68,1.0,-7.820852,2133.41,2.0,9.603405,2401.62,-2.636970,1792.65,30.437620
4,2017-07-14,2338.29,2355.02,2148.95,2178.06,2536.68,2.0,-14.137376,2133.41,3.0,2.092894,2408.58,-9.570784,1792.65,21.499456


In [6]:
crypto_pair3 = "ETH/USDT"
start_date3 =  start_date
df3 = fetch_crypto_data(crypto_pair= crypto_pair3 , start_date= start_date3)
df3 = calculate_metrics(df3 , 7 , 5)
df3.head()

,Date,Open,High,Low,Close,High_Last_7_Days,Days_Since_High_Last_7_Days,%_Diff_From_High_Last_7_Days,Low_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,%_Diff_From_High_Next_5_Days,Low_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2017-07-10,235.20,222.93,195.01,205.94,222.93,0.0,-7.621226,195.01,0.0,5.604841,180.27,14.239751,169.77,21.305295
1,2017-07-11,205.94,210.02,173.41,191.01,222.93,1.0,-14.318396,173.41,0.0,10.149357,180.27,5.957730,142.57,33.976292
2,2017-07-12,191.01,239.92,202.55,238.83,239.92,0.0,-0.454318,173.41,1.0,37.725621,189.52,26.018362,142.57,67.517711
3,2017-07-13,238.83,220.78,194.50,205.21,239.92,1.0,-14.467322,173.41,2.0,18.338043,256.06,-19.858627,142.57,43.936312
4,2017-07-14,205.21,195.85,181.13,194.09,239.92,2.0,-19.102201,173.41,3.0,11.925494,256.06,-24.201359,142.57,36.136635


In [7]:
crypto_pair4 = "ETH/USD"
start_date4 = start_date
df4 = fetch_crypto_data(crypto_pair4 , start_date4)
df4 = calculate_metrics(df4 , 7, 5)
df4.head()


,Date,Open,High,Low,Close,High_Last_7_Days,Days_Since_High_Last_7_Days,%_Diff_From_High_Last_7_Days,Low_Last_7_Days,Days_Since_Low_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,%_Diff_From_High_Next_5_Days,Low_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2017-07-10,237.72,240.09,188.21,205.76,240.09,0.0,-14.298805,188.21,0.0,9.324691,198.71,3.547884,168.95,21.787511
1,2017-07-11,205.76,216.16,175.21,190.55,240.09,1.0,-20.633929,175.21,0.0,8.755208,198.71,-4.106487,132.64,43.659530
2,2017-07-12,190.55,227.06,181.62,224.15,240.09,2.0,-6.639177,175.21,1.0,27.932196,198.71,12.802577,132.64,68.991255
3,2017-07-13,224.15,226.66,193.20,205.41,240.09,3.0,-14.444583,175.21,2.0,17.236459,256.89,-20.039706,132.64,54.862786
4,2017-07-14,205.41,207.07,182.62,197.14,240.09,4.0,-17.889125,175.21,3.0,12.516409,256.89,-23.258982,132.64,48.627865


In [8]:
# checking the Shape of all The fetched Dataframes to Ensure Data Consistency 
print(df1.shape , df2.shape , df3.shape , df4.shape) 


(2001, 15) (2001, 15) (2001, 15) (2001, 15)


As the Shape of All the DataFrames is Shape means that the Data Fetched is correct 

In [9]:
# Save DataFrames to an Excel file with different sheets
with pd.ExcelWriter("Fetched_crypto_data.xlsx") as writer:
    df1.to_excel(writer, sheet_name="Sheet1", index=False)
    df2.to_excel(writer, sheet_name="Sheet2", index=False)
    df3.to_excel(writer, sheet_name="Sheet3", index=False)
    df4.to_excel(writer, sheet_name="Sheet4", index=False)